In [12]:
import os
import sqlite3

import pandas as pd


# Data Cleaning and Analysis
'Carotid endarterectomy'

In [ ]:
cloud pricing...
ml: the study of tools and methods for identifying patterns in data
    
increase our understanding
make predictions

In [ ]:
This problem: target monitoring/prevention/rehab efforts

In [ ]:
Files:
  
EMR
- age
- gender
- race
- postal code
- diabetes
- immunosuppression


- medications

EMR section appended
- pain in limbs
- cold in limbs
- vision changes
- 

National Income Database
- Income

Waveform files of bp

claims medications

In [ ]:
# Collected Data

# 1. EHR tables in a relational database. Stored in `ehr/ehr.db`
patients = ['patient_id', 'name', 'dob', 'gender', 'race', 'postal_code', 'occupation', 'phone']

admissions = ['patient_id', 'hadm_id', 'intime', 'outtime']

procedures = ['hadm_id', 'icd9_code', 'clinician_id']

diagnoses = ['hadm_id', 'icd9_code', 'clinician_id']

chartevents = ['hadm_id', 'item_id', 'value', 'datetime']

d_procedures = {3000:'Carotid endarterectomy'}
d_diagnoses = {10000:'diabetes', 10001:'hiv', 10002:'cancer'}
d_chartevents = {2000:'weight', 2001:'abp'}


# 2. Streamed blood pressure numberics from wearable device.
# Stored in csv files: `blood-pressure/<patient_id>-<date>.csv`.
# Aggregated hourly data. Each patients have recordings both before and after their procedure.

# example format:
"""
hour,bp_min,bp_max
0,70,72
...
23,45,60
"""

# 3. Pharmacy orders. Stored in text files: pharmacy/NNNNNN.txt

# example format:
"""
Date:YYYY-MM-DD
Patient ID: 
Pharmacy: 
Drug: dexamethasone/erlotinib/other
"""



In [ ]:
# Research database tables

# Removed name, phone
patients = ['patient_id', 'dob', 'gender', 'race', 'postal_code', 'occupation'] 

admissions = ['patient_id', 'hadm_id', 'intime', 'outtime']

# Removed clinician_id
procedures = ['hadm_id', 'icd9_code'] 

# Removed clinician_id
diagnoses = ['hadm_id', 'icd9_code']

chartevents = ['hadm_id', 'item_id', 'value', 'datetime']

devicestream = ['patient_id', 'datetime', 'bp_min', 'bp_max']

medications = ['patient_id', 'datetime', 'drug']


## Task: Combine the disparate data sources to create a single usable sharable research database

In [24]:
# os.remove('data/allhealth.db')
conn = sqlite3.connect('data/allheath.db')
c = conn.cursor()

In [34]:
# Step 1 - Transfer the EHR data into the combined research database

# Read the data from the EHR
conn_ehr = sqlite3.connect('data/example-data-lake/ehr/ehr.db')

df_patients = pd.read_sql_query('select * from patients;', conn_ehr)
df_admissions = pd.read_sql_query('select * from admissions;', conn_ehr)
df_procedures = pd.read_sql_query('select * from procedures;', conn_ehr)
df_diagnoses = pd.read_sql_query('select * from diagnoses;', conn_ehr)
df_chartevents = pd.read_sql_query('select * from chartevents;', conn_ehr)

conn_ehr.close()

# Remove the unwanted columns
del(df_patients['name'])
del(df_patients['phone'])

del(df_procedures['clinician_id'])
del(df_diagnoses['clinician_id'])


In [37]:
df_chartevents.head()

,hadm_id,item_id,value,datetime
0,50001,2000,102,2018-01-01 02:00:00
1,50001,2001,50,2018-01-01 03:00:00
2,50001,2001,75,2018-01-01 04:00:00
3,50002,2000,100,2018-01-01 02:00:00
4,50002,2001,22,2018-01-01 03:00:00


In [ ]:

# Insert data
df_patients.to_sql('patients', conn, index=False)
df_admissions.to_sql('admissions', conn, index=False)
df_procedures.to_sql('procedures', conn, index=False)
df_diagnoses.to_sql('diagnoses', conn, index=False)
df_chartevents.to_sql('chartevents', conn, index=False)

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [ ]:


os.remove('data/allhealth.db')
conn = sqlite3.connect('data/allheath.db')
c = conn.cursor()

# Create tables
c.execute('''CREATE TABLE patients
             (patient_id integer primary key, name text, dob date, gender text,
              race text, postal_code integer, occupation text, phone integer);''')

c.execute('''CREATE TABLE admissions
             (hadm_id integer primary key, patient_id integer,
              intime datetime, outtime datetime);''')

c.execute('''CREATE TABLE procedures
             (hadm_id integer, icd9_code integer, clinician_id integer);''')

c.execute('''CREATE TABLE diagnoses
             (hadm_id integer, icd9_code integer, clinician_id integer);''')

c.execute('''CREATE TABLE chartevents
             (hadm_id integer, item_id integer, value integer, datetime datetime);''')


# Insert data
df_patients.to_sql('patients', conn, index=False, if_exists='append')
df_admissions.to_sql('admissions', conn, index=False, if_exists='append')
df_procedures.to_sql('procedures', conn, index=False, if_exists='append')
df_diagnoses.to_sql('diagnoses', conn, index=False, if_exists='append')
df_chartevents.to_sql('chartevents', conn, index=False, if_exists='append')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [3]:
# Final desired features (and outcome)
            # patients
features = ['age', 'gender', 'race', 'postal_code', 'occupation',
            # diagnoses
            'diabetes', 'hiv', 'cancer',
            # chartevents
            'weight_hosp', 'bp_hosp_min', 'bp_hosp_max',
            # bp numerics from wearables
            'bp_disch_min', 'bp_disch_max',
            # drug claims
            'dexamethasone', 'erlotinib'
            # symptoms from followup chat
            'limb_pain', 'limb_cold', 'vision_change']

outcome = ['readmission']


Q. Write scripts to map the raw data into a set of tables for a reusable research database.

Q. Write scripts to get data from the research database into the desired feature
   matrix for each patient for this study.
   
   
   
Q. Create a reusable research database from the raw data. (Even the relational db. You generally really be using the raw ehr for studies. You may, but in this workshop, we will create a new one.)

Q. Extract the desired features for the study cohort from the created research database.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features[:, :-1], features[:, -1],
                                                    train_size=0.75, test_size=0.25,
                                                    random_state=0)
print('Number of training records: %d' % len(x_train))
print('Number of testing records: %d' % len(x_test))


NameError: name 'features' is not defined

In [6]:
from sklearn import svm, neighbors
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
# LR
clf_lr = LogisticRegression()
clf_lr.fit(x_train, y_train)
y_predict_lr = clf_lr.predict(x_test)

# KNN
clf_knn = neighbors.KNeighborsClassifier()
clf_knn.fit(x_train, y_train)
y_predict_knn = clf_knn.predict(x_test)

# SVM
clf_svm = svm.SVC()
clf_svm.fit(x_train, y_train)
y_predict_svm = clf_svm.predict(x_test)

# And GB
clf_gb = GradientBoostingClassifier()
clf_gb.fit(x_train, y_train)
y_predict_gb = clf_gb.predict(x_test)

In [ ]:
def confusion_matrix_df(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm = pd.DataFrame(cm, columns=['Predict 0', 'Predict 1'], index=['Actual 0', 'Actual 1'])
    return cm

In [ ]:
print('Logistic Regression')
print(classification_report(y_test, y_predict_lr, target_names=['No readmission', 'Readmission']))

print('K Nearest Neigbors')
print(classification_report(y_test, y_predict_knn, target_names=BEAT_TYPES))

print('Support Vector Machines')
print(classification_report(y_test, y_predict_svm, target_names=BEAT_TYPES))

print('Gradient Boosting')
print(classification_report(y_test, y_predict_gb, target_names=BEAT_TYPES))


- encouraging people to give bad/false data
  - ie. falling vs dropping, yes vs no prior issues
  - misrepresenting information
  - no followup. initial diagnosis only.
  - Poor description of injury cause
  - providers stating they are giving treatments they are not due to pay by treatment
 

- 150000 claims costing most of the money. Soft tissue injuries. If off work for >3mo, not soft tissue.

- Need clinical/ehr info.

- Need social data. We don't capture now. Rehab is confounded by home environment. Psychosocial issues or physical issues? Why would provider tell us these issues?

- What is ACC's role in addressing these issues, as opposed to just paying for treatment?

- many clinical reports come in scanned pdf.

- Clinical notes given upon request. Specialist report, high-tech imaging. Treatment injury. Mostly up to ACC discretion. ie. ACL rupture, only request data if they request surgery or are off for a long time.

- We tell gp not to put in provisional initial diagnosis. May get declined. GP authorized or not to make certain diagnoses?

- We want continuous cases with more and more info. Want the ability to update cases.



Goals:
- good injury prevention, treatment rehab, compensation. Outcomes?

Semi supervised?
Trust issues of claims be approved/rejected by algorithms

